In [ ]:
from IPython.core.display import HTML
HTML('''
<script>
var logoParent = document.getElementById("kernel_logo_widget")
var logo = document.getElementById("kernel_logo_widget").getElementsByClassName("current_kernel_logo")[0];
logo.src = "https://i.ibb.co/mD4jTGQ/itclogo.jpg";
logo.style = "display: inline; width:138px; height:40px";
logoParent.innerHTML = '<a href="https://i.ibb.co/mD4jTGQ/itclogo.jpg">' + logoParent.innerHTML + '</a>';
</script>
''')

<font size="36"><b>Pandas - Part IV</b></font> <img src = "https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/03/pandas.jpg" height=200 width=200>

# Aggregation and grouping

An essential piece of analysis of large data is efficient summarization: computing aggregations like `sum()`, `mean()`, `median()`, `min()`, and `max()`, in which a single number gives insight into the nature of a potentially large dataset. In this section, we'll explore aggregations in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a `groupby`.

for the purpose of this exercise we'll use titanic survivor data.

download the file from http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls, and read it as a pandas dataframe

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls', index_col=None)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Column description:
- survival: Survival (0 = no; 1 = yes)
- pclass: Passenger class (1 = first; 2 = second; 3 = third)
- name: Name
- sex: Sex
- age: Age
- sibsp: Number of siblings/spouses aboard
- parch: Number of parents/children aboard
- ticket: Ticket number
- fare: Passenger fare
- cabin: Cabin
- embarked: Port of embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
- boat: Lifeboat (if survived)
- body: Body number (if did not survive and body was recovered)

A large dataset is great but to derive insights we need to summarize or reduce it in some way. We've already discussed various functions that achieve this: `mean`, `sum`, etc. Pandas contains a convenience method `describe` to achieve most of this in one line of code. Try it now on the titanic dataset.

In [4]:
df.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


While useful, we may want to have a finer understanding of what exactly is happening. Let's dig in to the differences based on cabin class (the pclass column). To do this we will use the `groupby` operator that split the dataframe into a list of dataframes by a given criterion.

In [5]:
df_gby_class = df.groupby('pclass')


Let's compute mean for all numeric columns by using operation on `df_gby_class`

In [6]:
df_gby_class.mean()

,survived,age,sibsp,parch,fare,body
pclass,,,,,,
1,0.619195,39.159918,0.436533,0.365325,87.508992,162.828571
2,0.429603,29.506705,0.393502,0.368231,21.179196,167.387097
3,0.255289,24.816367,0.568406,0.400564,13.302889,155.818182


**Notice how `pclass` became an index.  Need to pay attention not to loose it (might need to return it to be a regular column) if we want to continue working with this DataFrame**

Let't compute mean survival rate per pclass by using operation on `df_gby_class`

In [7]:
df_gby_class['survived'].mean()

pclass
1    0.619195
2    0.429603
3    0.255289
Name: survived, dtype: float64

After grouping we can access an individual group with `get_group`. Compute the mean survival rate for pclass 1 and 3

In [8]:
print('the mean survival rate for pclass 1 is {}\n'.format(df_gby_class['survived'].get_group(1).mean()))
print('the mean survival rate for pclass 3 is {}\n'.format(df_gby_class['survived'].get_group(3).mean()))

the mean survival rate for pclass 1 is 0.6191950464396285

the mean survival rate for pclass 3 is 0.2552891396332863



But, that's not where groupby shines. It shines when we use it to
1. split to groups
2. peform an action on each group
3. recombine the dataframe

## aggregation
aggregation allows us to perform a series of operations and present them in a dataframe. Here check this out

In [9]:
df.groupby('pclass')['survived'].aggregate([np.min, np.max, np.mean, np.median])

,amin,amax,mean,median
pclass,,,,
1,0,1,0.619195,1
2,0,1,0.429603,0
3,0,1,0.255289,0


## filtration
you can filter out using a call to the filter function with function which transforms a dataframe to a boolean. 

Let's prepare and compute the `std` by column per group:

In [10]:
df.groupby('pclass').std()

,survived,age,sibsp,parch,fare,body
pclass,,,,,,
1,0.486338,14.548059,0.609064,0.715602,80.447178,82.652172
2,0.495915,13.638628,0.590100,0.692717,13.607122,107.077753
3,0.436331,11.958202,1.299681,0.981639,11.494358,102.403720


Now let's get of any group with a std of less than 0.7 in the number of parents and/or children on board `parch` this will remove pclass 2

In [11]:
# filtering
filt07 = df.groupby('pclass').filter(lambda group: group['parch'].std() >= 0.7)
filt07

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN




**Notice** that `filter` returned not the groups, but the actual original rows after filtering based on grouped `std` function

In [12]:
# let's check that pclass 2 was indeed removed
len(filt07['pclass'])

1032

Now groupby the number of parents and/or children on board `parch` and remove groups for which the average `age` is less than 30.

In [19]:
rem = df.groupby('parch').filter(lambda group: group['age'].mean() < 30)
(rem['age'].mean())



22.704928793774318

The groupby allows you also to:
1. transform - apply same transformation to all columns in each group
2. apply - apply an arbitrary function to each group

<div class="alert alert-info">
<b>Note:</b>
<code>apply</code> implicitly passes all the columns for each group separately as a <code>DataFrame</code> to the custom function, while <code>transform</code> passes each column for each group as a Series to the custom function.
    
The custom function passed to <code>apply</code> can return a scalar, or a <code>Series</code> or <code>DataFrame</code> (or <code>numpy</code> array or even list). The custom function passed to <code>transform</code> must return a sequence (a one dimensional <code>Series</code>, array or list) the same length as the group.
So, <code>transform</code> works on just one <code>Series</code> at a time and <code>apply</code> works on the entire <code>DataFrame</code> at once.
</div>

Let's look at an example:

In [14]:
# Adding mean of specific column for specific group from the value
df.groupby('pclass').transform(lambda x: x + x.mean())

,survived,age,sibsp,parch,fare,body
0,1.619195,68.159918,0.436533,0.365325,298.846492,NaN
1,1.619195,40.076618,1.436533,2.365325,239.058992,NaN
2,0.619195,41.159918,1.436533,2.365325,239.058992,NaN
3,0.619195,69.159918,1.436533,2.365325,239.058992,297.828571
4,0.619195,64.159918,1.436533,2.365325,239.058992,NaN
...,...,...,...,...,...,...
1304,0.255289,39.316367,1.568406,0.400564,27.757089,483.818182
1305,0.255289,NaN,1.568406,0.400564,27.757089,NaN
1306,0.255289,51.316367,0.568406,0.400564,20.527889,459.818182
1307,0.255289,51.816367,0.568406,0.400564,20.527889,NaN


In [15]:
# Performing some operation. like adding mean and median of 2 different columns per group
df.groupby('pclass').apply(lambda gr: gr['parch'].mean() + gr['sibsp'].median())

pclass
1    0.365325
2    0.368231
3    0.400564
dtype: float64

In [16]:
df.groupby('parch').apply(lambda gr: gr['age'].sum()/gr['fare'].sum())

parch
0    0.931566
1    0.469206
2    0.265527
3    0.448445
4    0.378775
5    1.215633
6    0.884861
9    0.000000
dtype: float64

Now `groupby` number of parents and/or children `parch` and for each group compute sum of `age` divided by sum of `fare` (in that group). 

<div class="alert alert-warning"><b>NOTE:</b> Don't use a lambda function </div>

In [17]:
df.groupby('parch')['age'].aggregate([np.sum])/df.groupby('parch')['fare'].aggregate([np.sum])

,sum
parch,
0,0.931566
1,0.469206
2,0.265527
3,0.448445
4,0.378775
5,1.215633
6,0.884861
9,0.000000
